# Abhijit Bhaduri: Talent, Leadership, and Skills Guru

### "Careers lie at the intersection of three forces: work, workers and workplaces." —via [Career 3.0: Six Skills You Must Have to Succeed](https://www.amazon.com/Career-3-0-Skills-Must-Succeed-ebook/dp/B0CLRR81TK)

### We think across these three spaces defined by Abhijit in the context of the generative AI era.

Instructions for getting cozy with this AI recipe are on the [GitHub page's README](https://aka.ms/CAIK-repo). 

# 🧑‍🍳 Recipe for AI-driven Career Disruption

- ~500 tokens from COMPLETION Pre-trained Foundation Model
 
First, create a simple "agent" that can serve as your copywriter assistant.

Then we create an art director assistant with a love for David Ogilvy's style of copywriting.

Let the two assistants collaborate with you.

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta3"
#r "nuget: System.Linq.Async, 6.0.1"

# Fire up a kernel for the first simple agent 🔥

In [ ]:
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;

// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Configure the two AI features: OpenAI Chat and DALL-E 2 for image generation
var builder = new KernelBuilder();

// gpt-3.5-turbo is used by default
//model = "gpt-3.5-turbo";
model = "gpt-4";

Console.WriteLine($"Using 🧱 Model: {model}");

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();

// Get AI service instance used to manage the user chat
var chatGPT = kernel.GetService<IChatCompletion>();

Let's add a utility function that makes output more legible on screen.

In [ ]:
public static void WrappedWriteLine(string text, int columns)
{
    if (string.IsNullOrEmpty(text) || columns <= 0)
    {
        Console.WriteLine(text);
        return;
    }

    var words = text.Split(' ');
    var sb = new StringBuilder();

    string currentLine = "";

    foreach (var word in words)
    {
        if ((currentLine + word).Length <= columns)
        {
            currentLine += (string.IsNullOrEmpty(currentLine) ? "" : " ") + word;
        }
        else
        {
            sb.AppendLine(currentLine);
            currentLine = word;
        }
    }

    if (!string.IsNullOrEmpty(currentLine))
    {
        sb.AppendLine(currentLine);
    }

    Console.Write(sb.ToString());
}

## 💬 Put the copywriter agent to work

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

var systemMessage = "You are a copywriter with ten years of experience and are known for brevity and a dry humor. You're laser focused on the goal at hand. Don't waste time with chit chat. The goal is to refine and decide on the best copy as an expert in the field.";

var chat = (OpenAIChatHistory)chatGPT.CreateNewChat(systemMessage);

var lastReply = "";

while (true)
{
    // 1. Ask the user for a message. The user enters a message.  Add the user message into the Chat History object.
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.Write($"User: {userMessage}");
    chat.AddUserMessage(userMessage);

    // 2. Send the chat object to AI asking to generate a response. Add the bot message into the Chat History object.
    string assistantReply = await chatGPT.GenerateMessageAsync(chat, new OpenAIRequestSettings());
    chat.AddAssistantMessage(assistantReply);

    // 3. Show the reply
    WrappedWriteLine($"\nCopywriter: {assistantReply}",120);
    lastReply = assistantReply;
}

## 🖼️ Put the art director agent to work

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

IKernel kernel2 = builder.Build();

// Get AI service instance used to manage the user chat
var chatGPT2 = kernel2.GetService<IChatCompletion>();

var systemMessage2 = "You are an art director who has opinions about copywriting born of a love for David Ogilvy. You're laser focused on the goal at hand. Don't waste time with chit chat. The goal is to refine and decide on the best copy as an expert in the field.";

var chat2 = (OpenAIChatHistory)chatGPT2.CreateNewChat(systemMessage2);

chat2.AddUserMessage($"{lastReply} What do you think of this copy?");
string artdirectoryReply = await chatGPT.GenerateMessageAsync(chat2, new OpenAIRequestSettings());
chat2.AddAssistantMessage(artdirectoryReply);

var lastReply2 = "";

WrappedWriteLine($"\nArt Director: {artdirectoryReply}", 120);
lastReply2 = artdirectoryReply;

## 👀 Let the two agents "work" with each other

In [ ]:
// run for a few iterations:
for(var i = 0; i < 3; i++) {
    chat.AddUserMessage($"{lastReply2}");
    string assistantReply = await chatGPT.GenerateMessageAsync(chat, new OpenAIRequestSettings());
    chat.AddAssistantMessage(assistantReply);
    WrappedWriteLine($"\nCopywriter: {assistantReply}",120);
    lastReply = assistantReply;    

    chat2.AddUserMessage($"{lastReply}");
    artdirectoryReply = await chatGPT.GenerateMessageAsync(chat2, new OpenAIRequestSettings());
    chat2.AddAssistantMessage(artdirectoryReply);
    WrappedWriteLine($"\nArt Director: {artdirectoryReply}", 120);
    lastReply2 = artdirectoryReply;
}


In [ ]:
chat2.AddUserMessage($"Provide a concise prompt in as few words as possible to generate an image that best captures the essence of the copy.");
artdirectoryReply = await chatGPT.GenerateMessageAsync(chat2, new OpenAIRequestSettings());
chat2.AddAssistantMessage(artdirectoryReply);
WrappedWriteLine($"\nArt Director's suggested prompt: {artdirectoryReply}", 120);

## 🔥 Fire up a kernel with an image generation model

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration; 
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Configure the three AI features: text embedding (using Ada), text completion (using DaVinci 3), image generation (DALL-E 2)
var builder = new KernelBuilder();

if(useAzureOpenAI)
{
    builder.WithAzureOpenAIImageGenerationService(azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAIImageGenerationService(apiKey, orgId);
}
   
var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetService<IImageGeneration>();

## ✨ Put the art director to work

In [ ]:
#r "nuget: SkiaSharp, 2.88.3"

In [ ]:
#!import ../config/SkiaUtils.cs

using System;
using System.Threading.Tasks;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using System.IO;
using System.Net.Http;

HttpClient httpClient = new HttpClient();

async Task DownloadImageAsync(string uri, string localPath)
{
    using (HttpResponseMessage response = await httpClient.GetAsync(uri))
    {
        response.EnsureSuccessStatusCode();

        using (Stream contentStream = await response.Content.ReadAsStreamAsync(), 
                fileStream = new FileStream(localPath, FileMode.Create, FileAccess.Write, FileShare.None))
        {
            await contentStream.CopyToAsync(fileStream);
        }
    }
}

Console.WriteLine(artdirectoryReply);

var imageDescription = artdirectoryReply;
var image = await dallE.GenerateImageAsync(imageDescription, 256, 256);

Console.WriteLine(imageDescription);
Console.WriteLine("Image URL: " + image);
string uniqueFileName = "abc-" + Guid.NewGuid().ToString() + ".png";
Console.WriteLine(uniqueFileName);
await DownloadImageAsync(image,uniqueFileName);


## 👯 This WORK result is from the collaboration of two AI WORKERS :+).

In [ ]:
await SkiaUtils.ShowImage(image, 256, 256);

## 🏭 Will it change the WORKPLACE?